In [45]:
import pandas as pd

In [46]:
df = pd.read_csv("train.csv")

In [47]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [48]:
df.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [50]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [51]:
df=df.fillna('')

In [52]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [53]:
df.columns

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

In [54]:
df=df.drop(['id', 'title', 'author'], axis=1)

In [55]:
df.head()

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1
3,Videos 15 Civilians Killed In Single US Airstr...,1
4,Print \nAn Iranian woman has been sentenced to...,1


In [56]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

In [57]:
port_stem=PorterStemmer()

In [58]:
port_stem

<PorterStemmer>

In [59]:
port_stem.stem("Hi thIs is RAHAT * % %@@@")

'hi this is rahat * % %@@@'

In [60]:
def stemming(content):
    con=re.sub('[^a-zA-Z]', ' ', content)
    con=con.lower()
    con=con.split()
    con=[port_stem.stem(word) for word in con if not word in stopwords.words('english')]
    con=' '.join(con)
    return con

In [61]:
stemming('Hi this is SALAHUDDIN RAHAT')

'hi salahuddin rahat'

In [62]:
df['text']= df['text'].apply(stemming)

In [63]:
x=df['text']
y=df['label']
y.shape

(20800,)

In [64]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train, y_test = train_test_split(x, y, test_size=0.20)

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect=TfidfVectorizer()
x_train=vect.fit_transform(x_train)
x_test=vect.transform(x_test)

In [66]:
x_test.shape

(4160, 98266)

In [67]:
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier()
model.fit(x_train, y_train)

DecisionTreeClassifier()

In [68]:
prediction=model.predict(x_test)
prediction

array([1, 1, 0, ..., 1, 0, 0], dtype=int64)

In [69]:
model.score(x_test, y_test)

0.8824519230769231

In [70]:
import pickle

In [71]:
pickle.dump(vect, open('vector.pkl', 'wb'))

In [72]:
pickle.dump(model, open('model.pkl', 'wb'))

In [73]:
vector_form=pickle.load(open('vector.pkl', 'rb'))

In [74]:
load_model=pickle.load(open('model.pkl', 'rb'))

In [75]:
def fake_news(news):
    news=stemming(news)
    input_data=[news]
    vector_form1=vector_form.transform(input_data)
    prediction = load_model.predict(vector_form1)
    return prediction

In [76]:
val=fake_news("""In these trying times, Jackie Mason is the Voice of Reason. [In this week’s exclusive clip for Breitbart News, Jackie discusses the looming threat of North Korea, and explains how President Donald Trump could win the support of the Hollywood left if the U. S. needs to strike first.  “If he decides to bomb them, the whole country will be behind him, because everybody will realize he had no choice and that was the only thing to do,” Jackie says. “Except the Hollywood left. They’ll get nauseous. ” “[Trump] could win the left over, they’ll fall in love with him in a minute. If he bombed them for a better reason,” Jackie explains. “Like if they have no transgender toilets. ” Jackie also says it’s no surprise that Hollywood celebrities didn’t support Trump’s strike on a Syrian airfield this month. “They were infuriated,” he says. “Because it might only save lives. That doesn’t mean anything to them. If it only saved the environment, or climate change! They’d be the happiest people in the world. ” Still, Jackie says he’s got nothing against Hollywood celebs. They’ve got a tough life in this country. Watch Jackie’s latest clip above.   Follow Daniel Nussbaum on Twitter: @dznussbaum """)

In [77]:
if val==[0]:
    print('reliable')
else:
    print('unreliable')

reliable
